In [ ]:
#   Cryptocurrency Forecasting (Transformer + TCN Hybrid)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, Dense, Conv1D, LayerNormalization
from tensorflow.keras.layers import MultiHeadAttention, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.callbacks import EarlyStopping

from google.colab import files
import io

# 1. Ask user to upload dataset

print("Please upload your cryptocurrency OHLCV dataset (CSV file)…")
uploaded = files.upload()

file_name = list(uploaded.keys())[0]
df = pd.read_csv(io.BytesIO(uploaded[file_name]))

print("Columns detected:", df.columns.tolist())
print(df.head())

# 2. Preprocessing

# Choose Close price automatically if present
if "Close" in df.columns:
    df = df[["Close"]]
else:
    df = df.iloc[:, -1:]   # last column

df = df.dropna()

# Normalize 0–1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)

# 3. Create sequences

SEQ_LEN = 60   # lookback 60 minutes/hours
BATCH = 32

generator = TimeseriesGenerator(
    df_scaled, df_scaled,
    length=SEQ_LEN, batch_size=BATCH
)

# Train–test split
train_size = int(len(generator) * 0.8)
train_gen = generator[:train_size]
test_gen = generator[train_size:]

# 4. Transformer + TCN Model

def transformer_block(x, num_heads=4, ff_dim=64, dropout=0.1):
    # Multi-head attention
    attn = MultiHeadAttention(num_heads=num_heads, key_dim=ff_dim)(x, x)
    attn = Dropout(dropout)(attn)
    x = LayerNormalization()(x + attn)

    # Feed Forward
    fwd = Dense(ff_dim, activation="relu")(x)
    fwd = Dense(x.shape[-1])(fwd)
    x = LayerNormalization()(x + fwd)

    return x


def build_model(seq_len):
    inp = Input(shape=(seq_len, 1))

    # ---------- TCN BLOCK ----------
    tcn = Conv1D(64, kernel_size=3, dilation_rate=1, padding='causal', activation="relu")(inp)
    tcn = Conv1D(64, kernel_size=3, dilation_rate=2, padding='causal', activation="relu")(tcn)
    tcn = Conv1D(64, kernel_size=3, dilation_rate=4, padding='causal', activation="relu")(tcn)

    # ---------- TRANSFORMER BLOCK ----------
    transformer = transformer_block(tcn)

    flat = Flatten()(transformer)
    out = Dense(1)(flat)

    model = Model(inputs=inp, outputs=out)
    model.compile(optimizer="adam", loss="mse")

    return model


model = build_model(SEQ_LEN)
model.summary()

# 5. Train

es = EarlyStopping(monitor="loss", patience=5, restore_best_weights=True)

history = model.fit(
    generator,
    epochs=25,
    callbacks=[es],
    verbose=1
)

# 6. Forecast next 1 step

last_seq = df_scaled[-SEQ_LEN:].reshape(1, SEQ_LEN, 1)
pred = model.predict(last_seq)

pred_price = scaler.inverse_transform(pred)[0][0]
actual_last = df.iloc[-1].values[0]

print("\n----------------------------------")
print("Last Actual Price:", actual_last)
print("Predicted Next Price:", pred_price)
print("----------------------------------")

# 7. Plot training loss

plt.plot(history.history['loss'])
plt.title("Training Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()
